In [ ]:
import pickle
import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.utils import to_categorical 

from sklearn.preprocessing import normalize
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
train=pd.read_csv("/content/drive/MyDrive/Book by Brain/seed_2 (1).csv")
Y=pd.DataFrame({'label':train['label']},index=None)

In [ ]:
train.drop(['Unnamed: 0','label'],axis=1,inplace=True)

In [ ]:
train['combine'] = train.values.tolist()

In [ ]:
for i in range(train['combine'].shape[0]):
    train['combine'][i]=np.asarray(train['combine'][i]) ##merge the data to form an array

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
data = []
label = []
for i in range(0,train.shape[0]):
    data.append(train['combine'][i])
    label.append(Y['label'][i])

In [ ]:
new_train=np.asarray(data)
new_train=new_train.reshape(new_train.shape[0],new_train.shape[1],1)

In [ ]:
label_new=np.asarray(label)

In [ ]:
label_new = to_categorical(label_new)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(new_train, label_new, test_size=0.1, random_state=42)

In [ ]:
model=load_model('/content/drive/MyDrive/Book by Brain/bbb_v1.h5')
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 310, 128)          512       
                                                                 
 batch_normalization (BatchN  (None, 310, 128)         512       
 ormalization)                                                   
                                                                 
 max_pooling1d (MaxPooling1D  (None, 155, 128)         0         
 )                                                               
                                                                 
 conv1d_1 (Conv1D)           (None, 155, 128)          49280     
                                                                 
 batch_normalization_1 (Batc  (None, 155, 128)         512       
 hNormalization)                                                 
                                                        

In [ ]:
df=pd.read_csv('/content/drive/MyDrive/Book by Brain/Books_100k.csv')

In [ ]:
y_pred=model.predict(x_test)

In [ ]:
y=np.zeros((y_pred.shape[0],5))
for i in range(y_pred.shape[0]):
    y[i]=tf.math.sigmoid(y_pred[i])

In [ ]:
emotion2genre_matrix=np.array(
[[10,13,29,6,17,21,28,6,10,28,9,15,8,11,10,17,8,16,14,23,23],                                  #disgust
 [6,12,54,4,6,28,30,8,8,52,11,16,9,14,11,15,11,38,13,14,14],                                   #fear
 [8,10,35,4,11,21,23,7,8,38,10,13,8,23,20,23,9,29,12,15,21],                                   #sad
 [34,25,35,9,40,40,50,24,27,33,13,37,18,19,13,22,10,40,40,51,56],                              #neutral
 [24,16,33,5,30,38,56,19,17,34,13,44,11,12,13,17,6,45,41,52,57]]                               #happy 
 )

In [ ]:
y.shape

(2917, 5)

In [ ]:
emotion2genre_matrix.shape

(5, 21)

In [ ]:
emotion2genre_matrix = normalize(emotion2genre_matrix, axis=1, norm='l2')

In [ ]:
y_pred=np.matmul(y,emotion2genre_matrix)

In [ ]:
y_pred.shape

(2917, 21)

In [ ]:
y_pred=normalize(y_pred, axis=1, norm='l2')

In [ ]:
y_1=y_pred[0]

In [ ]:
ones = np.ones((1055, 21))

In [ ]:
y_1=y_1*ones

In [ ]:
genres=["Business", "Classics", "Comics", "Contemporary", "Crime", "Fantasy", 
        "Fiction","History", "Horror", "Humor", "Manga", "Mystery",
        "Nonfiction","Philosophy", "Poetry","Psychology", "Religion",
        "Romance", "Science", "Suspense","Thriller"
        ]

In [ ]:
vectors = df[genres].values

In [ ]:
normalized_vectors = normalize(vectors, axis=1, norm='l2')

In [ ]:
similarity = cosine_similarity(y_1, normalized_vectors)

In [ ]:
similarity

array([[0.09819441, 0.15442688, 0.27221257, ..., 0.18208721, 0.17102905,
        0.31338198],
       [0.09819441, 0.15442688, 0.27221257, ..., 0.18208721, 0.17102905,
        0.31338198],
       [0.09819441, 0.15442688, 0.27221257, ..., 0.18208721, 0.17102905,
        0.31338198],
       ...,
       [0.09819441, 0.15442688, 0.27221257, ..., 0.18208721, 0.17102905,
        0.31338198],
       [0.09819441, 0.15442688, 0.27221257, ..., 0.18208721, 0.17102905,
        0.31338198],
       [0.09819441, 0.15442688, 0.27221257, ..., 0.18208721, 0.17102905,
        0.31338198]])

In [ ]:
df['similarity'] = similarity[0]

In [ ]:
books= df.sort_values(by=['similarity', 'rating'], ascending=False) \
            .head(20)['title'] \
            .sample(frac=0.5)

In [ ]:
book_1=books.iloc[0]
book_2=books.iloc[1]
book_3=books.iloc[2]
book_4=books.iloc[3]
book_5=books.iloc[4]

In [ ]:
print(book_1)
print(book_2)
print(book_3)
print(book_4)
print(book_5)

Fallen Son: The Death of Captain America
Nightwing: Love and Bullets
Nightwing: The Hunt for Oracle
Kebakaran Hutan
Captain America by Ed Brubaker Omnibus, Vol. 1
